In [9]:
import requests
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
import json
import time

In [8]:
#Opening a Chrome browser
nos_deputes = webdriver.Chrome(r'chromedriver.exe')

#Open the page with the list of the national assembly folders
nos_deputes.get('https://www.nosdeputes.fr/dossiers/date')

In [3]:
#Getting all of the folders in the page
travaux = nos_deputes.find_element_by_class_name('travaux_parlementaires')

In [4]:
lst_dossiers = []
#Getting a list of selenium elements with all folders
dossiers = travaux.find_elements_by_tag_name('a')
#Putting the link of each folder in a list 
for dossier in dossiers:
    href = dossier.get_attribute('href')
    #Handling the case when there is simply no link
    if href is not None:
        lst_dossiers.append(href)

In [6]:
#Creating the dictionary that will take all the informations
dict_dossier = {}
#Instanciate counts to be able to control the flow and get back to a specific point if it crashes
count = 0
real_count = 0
#We iterate through each link, beginning from the index given by real_count
for dossier in lst_dossiers[real_count:]:
    #We put a "try" statement so that if something doesn't exist to handle these cases
    try:
        #The browser uses the link to the folder
        nos_deputes.get(dossier)
        #It searches for the title of the folder
        nom_dossier = nos_deputes.find_element_by_tag_name('h1').text
        #Creating a child index in the dictionary with the name of the folder
        dict_dossier[nom_dossier] = {}
        #Find the element containing the sessions in the page
        div_seances = nos_deputes.find_element_by_class_name('seances_dossier')
        lst_seances = []
        #Get a list with all the sessions
        seances = div_seances.find_elements_by_tag_name('a')
    #If the link to the folder doesn't work, it refreshes and go through the loop again, because real_count didn't change in the meantime
    except:
        nos_deputes.refresh()
        continue
    
    #For all sessions, we get the link in a list
    for seance in seances:
        href = seance.get_attribute('href')
        if href is not None:
            lst_seances.append(href)
     
    #For each link the list, we get to the link and wait 3 seconds so the page has time to load
    for seance in lst_seances:
        nos_deputes.get(seance)
        time.sleep(3)
        
        #If the info is not found, the loop repeats (get to the link and wait 3 seconds again), that prevents browser crashes
        try :
            #We print the session id so that we can follow the progression of the process
            id_seance = nos_deputes.find_element_by_class_name('titre_int_et_seance').find_element_by_tag_name('h1').text
            print(id_seance)
        except:
            continue
        
        #If the info is not found, the loop repeats (get to the link and wait 3 seconds again), that prevents browser crashes
        try:
            #We get all the tags
            tags = nos_deputes.find_element_by_class_name('nuage_de_tags').find_elements_by_tag_name('li')
            lst_tag = []
            for tag in tags:
                lst_tag.append(tag.text)
        except:
            lst_tag = []
        
        #We find all the parts containing interventions during the sessiion
        interventions = nos_deputes.find_elements_by_class_name('intervenant')
            
        lst_interventions = []
        
        #Will be used as an index for the next sub-dictionary
        num_intervention = 1
        #For each of these intervention, we put the relevant informations in the dict
        for intervention in interventions:
            try:
                #We get the speaker's name that sometimes include a title
                intervenant = intervention.find_element_by_class_name('perso').text
                
                #We handle the case when a title follows the speaker's name. If there is no title, it's "simply" a deputy
                intervenant_split = intervenant.split(',')
                if len(intervenant_split) > 1:
                    nom_intervenant = intervenant_split[0]
                    fonction_intervenant = intervenant_split[1:][0][1:]
                else:
                    nom_intervenant = intervenant_split[0]
                    fonction_intervenant = "député"
                
                #We get the intervention content with a list of each paragraph
                texte_intervention = intervention.find_element_by_class_name('texte_intervention')
                texte = texte_intervention.find_elements_by_tag_name('p')
                #We iterate through these list so that we only end up with one string
                texte_brut = ''
                for i in texte:
                    texte_brut += ' ' + i.text
                
                #We instanciate a list with each intervention as a dictionary including all the useful informations
                lst_interventions.append({num_intervention : str(num_intervention),
                                    'intervenant' : nom_intervenant,
                                  'fonction_intervenant' : fonction_intervenant,
                                  'texte' : texte_brut})
                num_intervention += 1
        
            except:
                    pass
         
        #We instanciate a session dictionary which is a child of the folder
        dict_dossier[nom_dossier][id_seance] = {'tags' : lst_tag,
                                              'interventions' : lst_interventions}
 
    count +=1
    real_count += 1
    
    #Every 10 folder, we save it in a json file
    if (count == 10) or (real_count == len(lst_dossiers)):
        nom_fichier = 'dossiers_' + str(real_count - 10) + '_' + str(real_count) + '.txt'
        with open(nom_fichier, 'w') as outfile:
            json.dump(dict_dossier, outfile)
        
        print('Fichier sauvegardé :' + nom_fichier)
        dict_dossier = {}
        count = 0
        
    else:
        pass
        print(real_count)

Séance en hémicycle du mardi 10 octobre 2017 à 15h00
531
Séance en hémicycle du mercredi 4 octobre 2017 à 15h00
532
Séance en hémicycle du mardi 3 octobre 2017 à 15h00
533
Séance en hémicycle du mercredi 9 août 2017 à 15h00
534
Séance en hémicycle du lundi 24 juillet 2017 à 16h00
Séance en hémicycle du lundi 24 juillet 2017 à 21h30
Séance en hémicycle du mardi 25 juillet 2017 à 15h00
Séance en hémicycle du mardi 25 juillet 2017 à 21h30
Séance en hémicycle du mercredi 26 juillet 2017 à 15h00
Séance en hémicycle du mercredi 26 juillet 2017 à 21h30
Séance en hémicycle du jeudi 27 juillet 2017 à 9h30
Séance en hémicycle du jeudi 27 juillet 2017 à 15h00
Séance en hémicycle du jeudi 27 juillet 2017 à 21h30
Séance en hémicycle du vendredi 28 juillet 2017 à 9h30
Séance en hémicycle du vendredi 28 juillet 2017 à 15h15
Séance en hémicycle du vendredi 28 juillet 2017 à 21h50
Séance en hémicycle du jeudi 3 août 2017 à 15h00
Séance en hémicycle du mercredi 9 août 2017 à 15h00
535
Séance en hémicycl